In [1]:
%AddJar file:///home/jovyan/work/apps/Emiasd-Flight-Data-Analysis.jar

Starting download from file:///home/jovyan/work/apps/Emiasd-Flight-Data-Analysis.jar
Finished download of Emiasd-Flight-Data-Analysis.jar
Using cached version of Emiasd-Flight-Data-Analysis.jar


In [2]:
import org.apache.spark.sql.SparkSession
import com.flightdelay.config.{AppConfiguration, ConfigurationLoader}
import com.flightdelay.data.loaders.FlightDataLoader

//Env Configuration
val args: Array[String] = Array("jupyter")

val spark = SparkSession.builder()
  .config(sc.getConf)
  .getOrCreate()

// Rendre la session Spark implicite
implicit val session = spark
implicit val configuration: AppConfiguration = ConfigurationLoader.loadConfiguration(args)

args = Array(jupyter)
spark = org.apache.spark.sql.SparkSession@246f4dd0
session = org.apache.spark.sql.SparkSession@246f4dd0
configuration = AppConfiguration(local,CommonConfig(42,true,debug,false,false,DataConfig(/home/jovyan/work/data,FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Flights/201201*.csv),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Weather/20101*.txt),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/wban_airport_timezone.csv)),OutputConfig(/home/jovyan/work/output,FileConfig(/home/jovyan/work/o...


AppConfiguration(local,CommonConfig(42,true,debug,false,false,DataConfig(/home/jovyan/work/data,FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Flights/201201*.csv),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Weather/20101*.txt),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/wban_airport_timezone.csv)),OutputConfig(/home/jovyan/work/output,FileConfig(/home/jovyan/work/o...

In [3]:
val rawFlightPath = s"${configuration.common.output.basePath}/common/data/processed_flights.parquet"
val rawFlightsDF = spark.read.parquet(rawFlightPath)

rawFlightPath = /home/jovyan/work/output/common/data/processed_flights.parquet
rawFlightsDF = [OP_CARRIER_AIRLINE_ID: int, DEST_AIRPORT_ID: int ... 30 more fields]


[OP_CARRIER_AIRLINE_ID: int, DEST_AIRPORT_ID: int ... 30 more fields]

In [4]:
print(rawFlightsDF.count())

3908461

In [5]:
rawFlightsDF.printSchema

root
 |-- OP_CARRIER_AIRLINE_ID: integer (nullable = true)
 |-- DEST_AIRPORT_ID: integer (nullable = true)
 |-- ORIGIN_AIRPORT_ID: integer (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- feature_arrival_time_period: string (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- feature_flight_week_of_year: integer (nullable = true)
 |-- feature_departure_time_period: string (nullable = true)
 |-- feature_departure_hour_rounded: long (nullable = true)
 |-- ARR_DELAY_NEW: double (nullable = true)
 |-- D1: integer (nullable = true)
 |-- D2_15: integer (nullable = true)
 |-- D2_30: integer (nullable = true)
 |-- D2_45: integer (nullable = true)
 |-- D2_60: integer (nullable = true)
 |-- D2_90: integer (nullable = true)
 |-- D3: integer (nullable = true)
 |-- D4: integer (nullable = true)
 |-- UTC_FL_DATE: date (nullable = true)
 |-- feature_utc_departure_hour_rounded: long (nullable = true)
 |-- UTC_ARR_DATE: date (nullable = true)
 |-- feature_utc_arriva

In [6]:
import org.apache.spark.sql.functions._
rawFlightsDF.select("WEATHER_DELAY").distinct().orderBy(asc("WEATHER_DELAY")).show()

+-------------+
|WEATHER_DELAY|
+-------------+
|         NULL|
|          0.0|
|          1.0|
|          2.0|
|          3.0|
|          4.0|
|          5.0|
|          6.0|
|          7.0|
|          8.0|
|          9.0|
|         10.0|
|         11.0|
|         12.0|
|         13.0|
|         14.0|
|         15.0|
|         16.0|
|         17.0|
|         18.0|
+-------------+
only showing top 20 rows



In [7]:
rawFlightsDF.select("NAS_DELAY").distinct().orderBy(asc("NAS_DELAY")).show()

+---------+
|NAS_DELAY|
+---------+
|     NULL|
|      0.0|
|      1.0|
|      2.0|
|      3.0|
|      4.0|
|      5.0|
|      6.0|
|      7.0|
|      8.0|
|      9.0|
|     10.0|
|     11.0|
|     12.0|
|     13.0|
|     14.0|
|     15.0|
|     16.0|
|     17.0|
|     18.0|
+---------+
only showing top 20 rows



In [8]:
val dxColumns = Seq("WEATHER_DELAY","NAS_DELAY")
val rawFlightsDFClean = rawFlightsDF.na.fill(0, dxColumns)


dxColumns = List(WEATHER_DELAY, NAS_DELAY)
rawFlightsDFClean = [OP_CARRIER_AIRLINE_ID: int, DEST_AIRPORT_ID: int ... 30 more fields]


[OP_CARRIER_AIRLINE_ID: int, DEST_AIRPORT_ID: int ... 30 more fields]

In [9]:
import com.flightdelay.data.preprocessing.flights.FlightDataSetFilterGenerator

val rawFlightsWithDxDF = FlightDataSetFilterGenerator.preprocess(rawFlightsDFClean, spark.emptyDataFrame, spark.emptyDataFrame)
print(rawFlightsWithDxDF.count())

[INFO] - Calling com.flightdelay.data.preprocessing.flights.FlightDataSetFilterGenerator.preprocess
[INFO] [FlightDataSetFilterGenerator] Total vols: 3908461
[INFO]  - D1     :   151694 vols (3.88%)
[INFO]  - D3     :   413953 vols (10.59%)
[INFO]  - D4     :  1433054 vols (36.67%)
[INFO]  - D2_15  :    11157 vols (0.29%)
[INFO]  - D2_30  :     5413 vols (0.14%)
[INFO]  - D2_45  :     2862 vols (0.07%)
[INFO]  - D2_60  :     1582 vols (0.04%)
[INFO]  - D2_90  :      593 vols (0.02%)
3908461

rawFlightsWithDxDF = [OP_CARRIER_AIRLINE_ID: int, DEST_AIRPORT_ID: int ... 30 more fields]


[OP_CARRIER_AIRLINE_ID: int, DEST_AIRPORT_ID: int ... 30 more fields]

In [10]:
rawFlightsWithDxDF.printSchema

root
 |-- OP_CARRIER_AIRLINE_ID: integer (nullable = true)
 |-- DEST_AIRPORT_ID: integer (nullable = true)
 |-- ORIGIN_AIRPORT_ID: integer (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- feature_arrival_time_period: string (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- feature_flight_week_of_year: integer (nullable = true)
 |-- feature_departure_time_period: string (nullable = true)
 |-- feature_departure_hour_rounded: long (nullable = true)
 |-- ARR_DELAY_NEW: double (nullable = true)
 |-- D1: integer (nullable = false)
 |-- D2_15: integer (nullable = false)
 |-- D2_30: integer (nullable = false)
 |-- D2_45: integer (nullable = false)
 |-- D2_60: integer (nullable = false)
 |-- D2_90: integer (nullable = false)
 |-- D3: integer (nullable = false)
 |-- D4: integer (nullable = false)
 |-- UTC_FL_DATE: date (nullable = true)
 |-- feature_utc_departure_hour_rounded: long (nullable = true)
 |-- UTC_ARR_DATE: date (nullable = true)
 |-- feature_ut

In [11]:
import java.sql.Date

val flightsColumns = Seq(
  "ARR_DELAY_NEW",
  "WEATHER_DELAY",
  "NAS_DELAY",  
  "D1",
  "D2_15",
  "D2_30",
  "D2_45",
  "D2_60",
  "D2_90",
  "D3",
  "D4"
)

val flightsData = rawFlightsWithDxDF
  .select(flightsColumns.head, flightsColumns.tail: _*)
  /**.filter(
    col("Date") === Date.valueOf("2012-01-09") &&
    (col("WBAN") === "13869" || col("WBAN") === "13874")
  )
  .orderBy("WBAN", "Date", "Time")**/
flightsData.show(10)

+-------------+-------------+---------+---+-----+-----+-----+-----+-----+---+---+
|ARR_DELAY_NEW|WEATHER_DELAY|NAS_DELAY| D1|D2_15|D2_30|D2_45|D2_60|D2_90| D3| D4|
+-------------+-------------+---------+---+-----+-----+-----+-----+-----+---+---+
|          0.0|          0.0|      0.0|  0|    0|    0|    0|    0|    0|  0|  0|
|          0.0|          0.0|      0.0|  0|    0|    0|    0|    0|    0|  0|  0|
|          3.0|          0.0|      0.0|  0|    0|    0|    0|    0|    0|  0|  1|
|          0.0|          0.0|      0.0|  0|    0|    0|    0|    0|    0|  0|  0|
|         49.0|          0.0|     49.0|  1|    0|    0|    0|    0|    0|  1|  1|
|          0.0|          0.0|      0.0|  0|    0|    0|    0|    0|    0|  0|  0|
|          0.0|          0.0|      0.0|  0|    0|    0|    0|    0|    0|  0|  0|
|         95.0|          0.0|     11.0|  0|    0|    0|    0|    0|    0|  1|  1|
|          3.0|          0.0|      0.0|  0|    0|    0|    0|    0|    0|  0|  1|
|          0.0| 

flightsColumns = List(ARR_DELAY_NEW, WEATHER_DELAY, NAS_DELAY, D1, D2_15, D2_30, D2_45, D2_60, D2_90, D3, D4)
flightsData = [ARR_DELAY_NEW: double, WEATHER_DELAY: double ... 9 more fields]


[ARR_DELAY_NEW: double, WEATHER_DELAY: double ... 9 more fields]

In [12]:
val flightDataCount = flightsData.count()
println(s"flightsData count: $flightDataCount")

flightsData count: 3908461


flightDataCount = 3908461


3908461

In [13]:
import org.apache.spark.sql.functions._

val countD2_60 = flightsData.filter(col("D2_60") === 1).count()
println(s"Nombre de lignes où D2_60 == 1 : $countD2_60")


Nombre de lignes où D2_60 == 1 : 1582


countD2_60 = 1582


1582

In [15]:
import com.flightdelay.features.balancer.DelayBalancedDatasetBuilder

val labeledFlightsData = DelayBalancedDatasetBuilder.prepareLabeledDataset(
    flightsData,
    "D2_60"
);



labeledFlightsData = [ARR_DELAY_NEW: double, WEATHER_DELAY: double ... 10 more fields]


[ARR_DELAY_NEW: double, WEATHER_DELAY: double ... 10 more fields]

In [16]:
val countis_Delayed = labeledFlightsData.filter(col("is_Delayed") === 1).count()
println(s"Nombre de lignes où is_Delayed == 1 : $countis_Delayed")

Nombre de lignes où is_Delayed == 1 : 1582


countis_Delayed = 1582


1582

In [17]:
import java.sql.Date

val flightsColumns = Seq(
  "ARR_DELAY_NEW",
  "D1",
  "D2_15",
  "D2_30",
  "D2_45",
  "D2_60",
  "D2_90",
  "D3",
  "D4",
  "is_Delayed"  
)

val labeledFlightsData2 = labeledFlightsData
  .select(flightsColumns.head, flightsColumns.tail: _*)
  /**.filter(
    col("Date") === Date.valueOf("2012-01-09") &&
    (col("WBAN") === "13869" || col("WBAN") === "13874")
  )
  .orderBy("WBAN", "Date", "Time")**/
labeledFlightsData2.show(10)

+-------------+---+-----+-----+-----+-----+-----+---+---+----------+
|ARR_DELAY_NEW| D1|D2_15|D2_30|D2_45|D2_60|D2_90| D3| D4|is_Delayed|
+-------------+---+-----+-----+-----+-----+-----+---+---+----------+
|          0.0|  0|    0|    0|    0|    0|    0|  0|  0|         0|
|          0.0|  0|    0|    0|    0|    0|    0|  0|  0|         0|
|          3.0|  0|    0|    0|    0|    0|    0|  0|  1|         0|
|          0.0|  0|    0|    0|    0|    0|    0|  0|  0|         0|
|         49.0|  1|    0|    0|    0|    0|    0|  1|  1|         0|
|          0.0|  0|    0|    0|    0|    0|    0|  0|  0|         0|
|          0.0|  0|    0|    0|    0|    0|    0|  0|  0|         0|
|         95.0|  0|    0|    0|    0|    0|    0|  1|  1|         0|
|          3.0|  0|    0|    0|    0|    0|    0|  0|  1|         0|
|          0.0|  0|    0|    0|    0|    0|    0|  0|  0|         0|
+-------------+---+-----+-----+-----+-----+-----+---+---+----------+
only showing top 10 rows



flightsColumns = List(ARR_DELAY_NEW, D1, D2_15, D2_30, D2_45, D2_60, D2_90, D3, D4, is_Delayed)
labeledFlightsData2 = [ARR_DELAY_NEW: double, D1: int ... 8 more fields]


[ARR_DELAY_NEW: double, D1: int ... 8 more fields]

In [18]:
val (trainDF, testDF) = DelayBalancedDatasetBuilder.buildBalancedTrainTest(labeledFlightsData, 0.75, 42L)

[DEBUG] Create balanced Train/Test datasets (50/50 delayed vs on-time) → Starting job: Create balanced Train/Test datasets (50/50 delayed vs on-time)
[INFO] [DEV  (trainBalanced)] total=    2392 | delayed=    1196 | on-time=    1196
[INFO] [TEST (testBalanced)] total=     772 | delayed=     386 | on-time=     386
[DEBUG] Create balanced Train/Test datasets (50/50 delayed vs on-time)  Completed in 8.48 s


trainDF = [ARR_DELAY_NEW: double, WEATHER_DELAY: double ... 10 more fields]
testDF = [ARR_DELAY_NEW: double, WEATHER_DELAY: double ... 10 more fields]


[ARR_DELAY_NEW: double, WEATHER_DELAY: double ... 10 more fields]